<a href="https://colab.research.google.com/github/Shanna2000/Investigate-Hotel-Business-using-Data-Visualization/blob/main/Hotel_Booking.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Import libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns

In [ ]:
sheet_id_1 = '1xq4t9GspOmGyXTnPpW8x8rh9oS3QYWJ9ZUT1YP5sVBI'

In [ ]:
xls =pd.ExcelFile(f"https://docs.google.com/spreadsheets/d/{sheet_id_1}/export?format=xlsx")
df = pd.read_excel(xls, 'hotel_bookings_data',header=0)

In [ ]:
df[df.columns[13:23]]

In [ ]:
df.head()

In [ ]:
df.shape

In [ ]:
df.info()

# Tugas 1 - Data  Preprocessing

## Null Value Handling

In [ ]:
df.isnull().sum()

Dari nilai di atas, maka dapat disimpulkan:
- Dikarenakan nilai null di kolom company sangat besar (lebih dari 90%), maka kolom 'company' akan dibuang
- Kolom agent yang null akan diganti menjadi 0
- Kolom city yang null akan diganti menjadi 'Kota Denpasar', modus kolom tersebut
- Kolom children yang null akan diganti menjadi 0, modus kolom tersebut

In [ ]:
df['city'].value_counts()

In [ ]:
#Membuang kolom 'company'
df.drop(columns = 'company', inplace = True)

In [ ]:
#Mengganti nilai null kolom 'agent' menjadi 0
df['agent'] = df['agent'].fillna(0)

#Mengganti nilai null kolom 'city' menjadi 'Kota Denpasar'
df['city'] = df['city'].fillna('Kota Denpasar')

#Mengganti nilai null kolom 'children' menjadi 0
df['children'] = df['children'].fillna(0)

In [ ]:
df.isnull().sum()

Nilai null pada data sudah ditangani

## Handle Nilai Tidak Wajar

In [ ]:
categorical = []
numerical = []
for i in df.columns:
  if df[i].dtype == 'object':
    categorical.append(i)
  else:
    numerical.append(i)

In [1]:
categorical

NameError: ignored

In [ ]:
for i in categorical:
  print(df[i].value_counts())

Terdapat beberapa kolom kategorikal yang memiliki nilai yang tak terdefinisi (Undefined) yang nantinya bisa mengecoh machine learning. Oleh karena itu:
- Untuk kolom 'meal' tidak dapat dihapus dikarenakan nilainya yang terlalu besar. Oleh karena itu, nilai Undefined akan diganti menjadi 'No Meal'.
- Kolom 'distribution_channel' dan 'market_segment' hanya memiliki jumlah yang sangat kecil value Undefined. Oleh karena itu dapat dibuang

In [ ]:
# Mengganti value undefined pada kolom 'meal' menjadi 'No Meal'
df['meal'] = df['meal'].replace('Undefined', 'No Meal')

In [ ]:
# Menghapus nilai undefined dari kolom 'distribution_channel' dan 'market_segment'
df = df[~((df['market_segment'] == 'Undefined') | (df['distribution_channel'] == 'Undefined'))]

In [ ]:
df['distribution_channel'].value_counts()

In [ ]:
df['market_segment'].value_counts()

# Tugas 2 - Data Visualization

In [ ]:
df

In [ ]:
df

In [ ]:
import math
group_df = df.groupby(['hotel', 'arrival_date_month'])['arrival_date_year'].agg(['nunique', 'count']).reset_index()
group_df['rata-rata per tahun'] = (group_df['count'] / group_df['nunique']).apply(lambda x: math.ceil(x))
group_df

In [ ]:
from datetime import datetime
# Define a function to convert month names to month numbers
def month_number(month_name):
    datetime_object = datetime.strptime(month_name, "%B")
    month_number = datetime_object.month
    return month_number

# Add a new column with the month number
group_df['Month_number'] = group_df['arrival_date_month'].apply(month_number)

In [ ]:
group_df = group_df.sort_values(['hotel', 'Month_number']).reset_index(drop = True)
group_df

In [ ]:
plt.figure(figsize=(15, 6))
sns.lineplot(data = group_df, x = 'arrival_date_month', y = 'rata-rata per tahun', hue = 'hotel')

In [ ]:
fig, ax = plt.subplots(2,1, figsize = (10,7))

sns.lineplot(data = group_df[group_df['hotel'] == 'City Hotel'], x = 'arrival_date_month', y = 'rata-rata per tahun', ax = ax[0], markers = True)
ax[0].set_title('Grafik Chart City Hotel')
ax[0].set_xlabel('Bulan')
ax[0].set_ylabel('Jumlah Pemesanan Hotel')

for _, row in group_df[group_df['hotel'] == 'City Hotel'].iterrows():
    ax[0].text(row['arrival_date_month'], row['rata-rata per tahun'], row['rata-rata per tahun'], ha='center', va='bottom')

sns.lineplot(data = group_df[group_df['hotel'] == 'Resort Hotel'], x = 'arrival_date_month', y = 'rata-rata per tahun', ax = ax[1], color = 'orange', markers = True)
ax[1].set_title('Grafik Chart Resort Hotel')
ax[1].set_xlabel('Bulan')
ax[1].set_ylabel('Jumlah Pemesanan Hotel')

for _, row in group_df[group_df['hotel'] == 'Resort Hotel'].iterrows():
    ax[1].text(row['arrival_date_month'], row['rata-rata per tahun'], row['rata-rata per tahun'], ha='center', va='bottom')

plt.tight_layout()

In [ ]:
fig, ax = plt.subplots(2,1, figsize = (10,7))

sns.lineplot(data = group_df[group_df['hotel'] == 'City Hotel'], x = 'arrival_date_month', y = 'rata-rata per tahun', ax = ax[0], markers = True)
ax[0].set_title('Grafik Chart City Hotel')
ax[0].set_xlabel('Bulan')
ax[0].set_ylabel('Jumlah Pemesanan Hotel')

sns.lineplot(data = group_df[group_df['hotel'] == 'Resort Hotel'], x = 'arrival_date_month', y = 'rata-rata per tahun', ax = ax[1], color = 'orange', markers = True)
ax[1].set_title('Grafik Chart Resort Hotel')
ax[1].set_xlabel('Bulan')
ax[1].set_ylabel('Jumlah Pemesanan Hotel')

plt.tight_layout()

Berdasarkan gambar, interpretasi yang didapatkan: <br>
<b>City Hotel</b>
- Pada bulan Maret - Juli jumlah pemesanan hotel mengalami kecenderungan <b>peningkatan</b> yang signifikan. Selama periode tersebut, hotel mengalami peningkatan rata-rata sekitar 120%.
- Pada bulan Januari - Maret, jumlah pemesanan hotel perlahan mengalami <b>penurunan</b> sekitar 14%
- Pada bulan Juli - September, jumlah pemesanan mengalami <b>penurunan</b> sebesar 34%
- Hotel kembali mengalami <b>peningkatan</b> di periode bulan September - Desember sebesar 41% <br>

<b>Resort Hotel</b>
- Pada bulan Maret - Juni jumlah pemesanan hotel mengalami kecenderungan <b>peningkatan</b> yang signifikan. Selama periode tersebut, hotel mengalami peningkatan rata-rata sekitar 64%
- Pada bulan Januari - Maret, jumlah pemesanan hotel perlahan mengalami <b>penurunan</b> sekitar 10%
- Pada bulan Juni - Agustus, jumlah pemesanan mengalami <b>penurunan</b> sebesar 16%
- Hotel kembali mengalami <b>peningkatan</b> di periode bulan Agustus - Desember sebesar 17% <br>